In [1]:
import os
import sys
sys.path.append(os.path.abspath(".."))

In [2]:
from belashovplot import TiledPlot
from utilities import *
from utilities.filters import Gaussian, Window
from utilities.training import train, confusion
from elements.modulators import Lens, PhaseModulator, AmplitudeModulator
from elements.propagators import FurrierPropagation, ConvolutionalPropagation
from elements.composition import CompositeModel, HybridModel
from elements.wrappers import CudaMemoryChunker, Incoherent
from elements.detectors import ClassificationDetectors, MatrixDetectors
from parameters import FigureWidthHeight, FontLibrary
from tqdm import tqdm
from math import sin, sqrt
from pickle import dump
import torch
import numpy
import matplotlib.pyplot as plt
import timm
from copy import deepcopy

C:\Users\uclap\Documents\Coding\Python\OpticalEncoder\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
FontLibrary.Fonts.PlotTitle.FontSize = 16

FontLibrary.Fonts.DescriptionLeft.FontSize = 12
FontLibrary.Fonts.DescriptionLeft.FontWeight = 'bold'
FontLibrary.Fonts.DescriptionLeft.FontStyle = 'italic'

FontLibrary.Fonts.DescriptionBottom.FontSize = 10
FontLibrary.Fonts.DescriptionBottom.FontWeight = 'bold'
FontLibrary.Fonts.DescriptionBottom.FontStyle = 'italic'

FontLibrary.Fonts.DescriptionTop.FontSize = 10
FontLibrary.Fonts.DescriptionTop.FontWeight = 'bold'
FontLibrary.Fonts.DescriptionTop.FontStyle = 'italic'

FontLibrary.Fonts.ColumnDescriptionTop.FontSize = 9
FontLibrary.Fonts.ColumnDescriptionBottom.FontSize = 9
FontLibrary.Fonts.RowDescriptionLeft.FontSize = 9
FontLibrary.Fonts.RowDescriptionRight.FontSize = 9
FontLibrary.Fonts.AxisX.FontSize = 8
FontLibrary.Fonts.AxisY.FontSize = 8
FigureWidthHeight = (6.69291, 10.1181-1.0)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Используемый девайс:', torch.cuda.get_device_name(device) if torch.cuda.is_available() else 'ЦП')

Используемый девайс: NVIDIA GeForce RTX 3060 Ti


# Смешанная лосс функция и инициализация ResNet

In [5]:
def combined_loss(cross_entropy_to_mse_proportion:float=1.0):
    def loss_function(outputs, targets):
        CELoss = torch.nn.functional.cross_entropy(outputs, targets)
        MSELoss = torch.nn.functional.mse_loss(torch.nn.functional.softmax(outputs, dim=1), torch.nn.functional.one_hot(targets, num_classes=10).float())
        loss = cross_entropy_to_mse_proportion*CELoss + (1.0-cross_entropy_to_mse_proportion)*MSELoss
        return loss
    return loss_function

In [6]:
def resnet18(classes:int=10):
    return timm.create_model('resnet18', pretrained=True,  in_chans=1, num_classes=classes).to(device)

# Обучение вариантов сети ResNet18

## Изображение 24x24

In [7]:
N = 24
batch_size = 22
learning_rate = 0.009854
loss_function = combined_loss(0.609798)
dataset = Dataset('CIFAR10', batch_size, N, N, torch.float32)
dataset.train
dataset.test
model = resnet18(10)
optimizer_type = torch.optim.RMSprop

Files already downloaded and verified
Files already downloaded and verified


In [8]:
loss_histories = []
cofusion_matrixes = []

In [9]:
optimizer = optimizer_type(model.parameters(), lr=learning_rate)
cofusion_matrixes.append(confusion(model, dataset))
print(f"Точность: {100*numpy.sum(numpy.diagonal(cofusion_matrixes[-1], 0))/numpy.sum(cofusion_matrixes[-1])}%")
for i in range(10):
    loss_histories.append(train(model, dataset, optimizer, loss_function))
    cofusion_matrixes.append(confusion(model, dataset))
    print(f"Точность: {100*numpy.sum(numpy.diagonal(cofusion_matrixes[-1], 0))/numpy.sum(cofusion_matrixes[-1])}%")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:08<00:00, 52.65it/s]


Точность: 13.29%


RLoss: 0.8231390216811834, RPI1000: -0.23457574294091552: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [00:51<00:00, 44.32it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:05<00:00, 83.88it/s]


Точность: 39.9%


RLoss: 0.8226104283589571, RPI1000: -0.038503313486092434: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [00:49<00:00, 45.78it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:05<00:00, 82.63it/s]


...

Точность: 68.79%


RLoss: 0.5199221544386816, RPI1000: 0.01627006787613319: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [00:47<00:00, 47.46it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:05<00:00, 85.59it/s]


Точность: 67.91%


RLoss: 0.34486891793982943, RPI1000: 0.011938303942878092: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [00:50<00:00, 44.89it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:05<00:00, 83.38it/s]


Точность: 67.55%


In [10]:
optimizer = optimizer_type(model.parameters(), lr=learning_rate/100)
for i in range(10):
    loss_histories.append(train(model, dataset, optimizer, loss_function))
    cofusion_matrixes.append(confusion(model, dataset))
    print(f"Точность: {100*numpy.sum(numpy.diagonal(cofusion_matrixes[-1], 0))/numpy.sum(cofusion_matrixes[-1])}%")

RLoss: 0.3456575933279774, RPI1000: -0.027236920963243316: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [00:54<00:00, 41.57it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:06<00:00, 73.34it/s]


Точность: 71.51%


RLoss: 0.2502756806370802, RPI1000: -0.004329742275493875: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [00:52<00:00, 43.22it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:05<00:00, 76.78it/s]


Точность: 71.31%


RLoss: 0.27147377883095286, RPI1000: -0.002821596220767629: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [00:55<00:00, 41.17it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:06<00:00, 74.13it/s]


...

Точность: 72.04%


RLoss: 0.18385037853112776, RPI1000: -0.0036689186524007274: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [00:53<00:00, 42.56it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:05<00:00, 80.16it/s]


Точность: 71.45%


RLoss: 0.2133598822804892, RPI1000: 0.00495361911882345: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [00:52<00:00, 43.19it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:05<00:00, 79.26it/s]

Точность: 71.42%


In [11]:
data1 = deepcopy((loss_histories, cofusion_matrixes, model))

## Изображение 32x32

In [15]:
N = 32
batch_size = 22
learning_rate = 0.009854
loss_function = combined_loss(0.609798)
dataset = Dataset('CIFAR10', batch_size, N, N, torch.float32)
dataset.train
dataset.test
model = resnet18(10)
optimizer_type = torch.optim.RMSprop

Files already downloaded and verified
Files already downloaded and verified


In [16]:
loss_histories = []
cofusion_matrixes = []

In [17]:
optimizer = optimizer_type(model.parameters(), lr=learning_rate)
cofusion_matrixes.append(confusion(model, dataset))
print(f"Точность: {100*numpy.sum(numpy.diagonal(cofusion_matrixes[-1], 0))/numpy.sum(cofusion_matrixes[-1])}%")
for i in range(10):
    loss_histories.append(train(model, dataset, optimizer, loss_function))
    cofusion_matrixes.append(confusion(model, dataset))
    print(f"Точность: {100*numpy.sum(numpy.diagonal(cofusion_matrixes[-1], 0))/numpy.sum(cofusion_matrixes[-1])}%")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:05<00:00, 77.21it/s]


Точность: 11.45%


RLoss: 0.930513141486318, RPI1000: -0.2675201930307615: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [00:51<00:00, 44.00it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:05<00:00, 81.39it/s]


Точность: 43.94%


RLoss: 0.6286695215243756, RPI1000: -0.06986399803143717: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [00:50<00:00, 44.84it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:05<00:00, 85.78it/s]


...

Точность: 68.29%


RLoss: 0.4103819580336504, RPI1000: 0.021151729377148785: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [00:48<00:00, 46.51it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:05<00:00, 85.30it/s]


Точность: 65.33%


RLoss: 0.40797999267120155, RPI1000: 0.012061517459817015: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [00:50<00:00, 45.38it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:05<00:00, 83.85it/s]

Точность: 68.97%


In [18]:
optimizer = optimizer_type(model.parameters(), lr=learning_rate/100)
for i in range(10):
    loss_histories.append(train(model, dataset, optimizer, loss_function))
    cofusion_matrixes.append(confusion(model, dataset))
    print(f"Точность: {100*numpy.sum(numpy.diagonal(cofusion_matrixes[-1], 0))/numpy.sum(cofusion_matrixes[-1])}%")

RLoss: 0.2860436281462009, RPI1000: -0.03435165850912686: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [00:48<00:00, 46.87it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:05<00:00, 90.75it/s]


Точность: 74.91%


RLoss: 0.2725527720943801, RPI1000: -0.008636469943095733: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [00:48<00:00, 47.23it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:05<00:00, 86.06it/s]


Точность: 75.48%


RLoss: 0.1722056413222925, RPI1000: -0.002870992614082347: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [00:49<00:00, 46.20it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:05<00:00, 80.85it/s]


...

Точность: 75.55%


RLoss: 0.1743388391728073, RPI1000: -0.0023069196773071806: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [00:49<00:00, 45.62it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:05<00:00, 82.36it/s]


Точность: 76.34%


RLoss: 0.182280005390646, RPI1000: -0.00359797950293199: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2273/2273 [00:51<00:00, 43.91it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:05<00:00, 83.76it/s]

Точность: 75.86%


In [19]:
data2 = deepcopy((loss_histories, cofusion_matrixes, model))